In [ ]:
# default_exp utils

In [ ]:
#export
from sensorai.imports import *
from sensorai.tf_imports import *

In [ ]:
#hide
from nbdev.showdoc import *

# Utility Belts
> Contain some common helper functions

## 1% Better Everyday

# Math 

In [ ]:
#export
def moving_average(x, window=5):
    """
    Return a moving-window average.
    :param x: a numpy array
    :param window: an integer, number of data points for window size.
    """
    return (pd.DataFrame(x)
                .rolling(window=window, min_periods=1)
                .mean()
                .values
                .squeeze())

In [ ]:
x = np.arange(10)
test_eq(moving_average(x), [0.,0.5,1.,1.5,2.,3.,4.,5.,6.,7.])
test_eq(moving_average(x,4), [0.,0.5,1.,1.5,2.5,3.5,4.5,5.5,6.5,7.5])